### Downloading Data from Data Portal using Login Credential 
#### Using Selenium - By RAHUL 

In [21]:
import os, time, csv, glob
from datetime import datetime, timedelta

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, StaleElementReferenceException

In [22]:
# ---------------- CONFIG ----------------
CSV_PATH = r"C:\Users\Administrator\LIS\MISAutomation\downloads\creds.csv"        # <-- change to your CSV path (A2: username, B2: password)
DOWNLOAD_DIR = os.path.abspath("./downloads")  # downloaded file target
BASE_URL = "https://appv2.loveinstore.com/Home"
MAX_WAIT_SECS = 45
HEADLESS       = True

# ----------------------------------------

os.makedirs(DOWNLOAD_DIR, exist_ok=True)


In [23]:
def read_creds(csv_path, row_index=1, col_user=0, col_pass=1):
    with open(csv_path, 'r', encoding='utf-8-sig', newline='') as f:
        sample = f.read(2048)
        f.seek(0)
        try:
            dialect = csv.Sniffer().sniff(sample)
        except Exception:
            dialect = csv.excel
        reader = csv.reader(f, dialect)
        rows = [[c.strip() for c in r] for r in reader if any(c.strip() for c in r)]

    if len(rows) <= row_index:
        raise ValueError(f"Need at least {row_index+1} non-empty rows; got {len(rows)}. Put username in A2, password in B2.")
    row = rows[row_index]
    if len(row) <= max(col_user, col_pass):
        raise ValueError(f"Row {row_index+1} has only {len(row)} columns: {row}. Expect username in col A, password in col B.")
    username, password = row[col_user], row[col_pass]
    if not username or not password:
        raise ValueError(f"Row {row_index+1} missing username/password values: {row}.")
    return username, password

username, password = read_creds(CSV_PATH)

# =========================
# CHROME (headless) + downloads
# =========================
chrome_options = Options()
if HEADLESS:
    chrome_options.add_argument("--headless=new")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--window-size=1920,1080")
chrome_options.add_argument("--ignore-certificate-errors")
chrome_options.add_argument("--allow-insecure-localhost")
chrome_options.add_argument("--ignore-ssl-errors=yes")

prefs = {
    "download.default_directory": DOWNLOAD_DIR,
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "safebrowsing.enabled": True,
}
chrome_options.add_experimental_option("prefs", prefs)

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)
wait    = WebDriverWait(driver, MAX_WAIT_SECS)
actions = ActionChains(driver)

# =========================
# HELPERS
# =========================
def js_click(el):
    driver.execute_script("arguments[0].click();", el)

def safe_click(locator, timeout=40, attempts=3):
    for attempt in range(attempts):
        el = WebDriverWait(driver, timeout).until(EC.element_to_be_clickable(locator))
        try:
            el.click()
            return el
        except Exception:
            try:
                js_click(el)
                return el
            except StaleElementReferenceException:
                if attempt == attempts - 1:
                    raise
                time.sleep(0.3)
    raise TimeoutException(f"Could not click {locator}")

def switch_into_frame_containing(locator, timeout=30):
    driver.switch_to.default_content()
    # already in default?
    try:
        WebDriverWait(driver, 3).until(EC.presence_of_element_located(locator))
        return False
    except TimeoutException:
        pass
    # search iframes
    frames = driver.find_elements(By.TAG_NAME, "iframe")
    for fr in frames:
        try:
            driver.switch_to.default_content()
            driver.switch_to.frame(fr)
            WebDriverWait(driver, 3).until(EC.presence_of_element_located(locator))
            return True
        except Exception:
            continue
    driver.switch_to.default_content()
    return False

def wait_for_url_contains(part, timeout=40):
    WebDriverWait(driver, timeout).until(lambda d: part in d.current_url)

def set_date_input(input_el, value_ddmmyyyy):
    driver.execute_script("arguments[0].removeAttribute('readonly');", input_el)
    input_el.clear()
    input_el.click()
    input_el.send_keys(value_ddmmyyyy)
    actions.move_by_offset(0, 120).click().perform()  # blur

def find_first(locators, timeout_each=20):
    last_err = None
    for loc in locators:
        try:
            return WebDriverWait(driver, timeout_each).until(EC.presence_of_element_located(loc))
        except Exception as e:
            last_err = e
    raise last_err

def wait_for_download(dir_path, timeout=240):
    start = time.time()
    seen = set(glob.glob(os.path.join(dir_path, "*")))
    while time.time() - start < timeout:
        time.sleep(1)
        current = set(glob.glob(os.path.join(dir_path, "*")))
        new_files = list(current - seen)
        if new_files:
            # wait out any .crdownload
            while any(p.endswith(".crdownload") for p in new_files):
                time.sleep(1)
                current = set(glob.glob(os.path.join(dir_path, "*")))
                new_files = list(current - seen)
            return max(new_files, key=os.path.getmtime)
    return None

# =========================
# MAIN FLOW
# =========================
try:
    print("Opening site…")
    driver.get(BASE_URL)

    # --- Login ---
    print("Logging in…")
    username_field = wait.until(EC.presence_of_element_located((By.ID, "UserName")))
    password_field = wait.until(EC.presence_of_element_located((By.ID, "Password")))
    login_button   = wait.until(EC.element_to_be_clickable((By.ID, "sign")))
    driver.execute_script("arguments[0].removeAttribute('readonly');", username_field)
    driver.execute_script("arguments[0].removeAttribute('readonly');", password_field)
    username_field.clear(); username_field.send_keys(username)
    password_field.clear(); password_field.send_keys(password)
    login_button.click()

    # wait for a post-login signal
    wait.until(EC.any_of(
        EC.presence_of_element_located((By.XPATH, "//aside|//div[contains(@class,'sidebar')]")),
        EC.presence_of_element_located((By.XPATH, "//a[contains(.,'Logout') or contains(@href,'Logout')]"))
    ))
    time.sleep(1.0)

    # --- Data Management ---
    print("Opening Data Management…")
    safe_click((
        By.XPATH,
        "//a[contains(@href,'/Admin/DataManagement/Index') and .//span[contains(text(),'Data Management')]]"
    ), timeout=40)

    # --- Attendance ---
    print("Opening Attendance…")
    attendance_locators = [
        (By.XPATH, "//a[contains(.,'Attendance') and not(contains(@href,'MonthlyAttendance'))]"),
        (By.XPATH, "//a[contains(@href,'/Attendance') and contains(.,'Attendance')]"),
    ]
    clicked = False
    for loc in attendance_locators:
        try:
            safe_click(loc, timeout=40)
            clicked = True
            break
        except Exception:
            continue
    if not clicked:
        raise TimeoutException("Could not open Attendance (all locators failed).")

    # --- Monthly Attendance ---
    print("Opening Monthly Attendance…")
    monthly_locators = [
        (By.XPATH, "//a[contains(@href,'/AttendanceV2/MonthlyAttendanceV2') and contains(.,'Monthly Attendance')]"),
        (By.XPATH, "//a[contains(.,'Monthly Attendance')]"),
    ]
    clicked = False
    for loc in monthly_locators:
        try:
            el = WebDriverWait(driver, 40).until(EC.presence_of_element_located(loc))
            driver.execute_script("arguments[0].scrollIntoView({block:'center'});", el)
            try:
                el.click()
            except Exception:
                js_click(el)
            clicked = True
            break
        except Exception:
            continue
    if not clicked:
        raise TimeoutException("Could not open Monthly Attendance (all locators failed).")

    # confirm navigation or probe DOM
    try:
        wait_for_url_contains("MonthlyAttendance", timeout=40)
    except Exception:
        pass

    # --- Page ready (iframe-aware) ---
    target_dropdown = (By.XPATH, "//*[@id='Sdatewise' or @name='Sdatewise' or @id='select2-Sdatewise-container']")
    switched = switch_into_frame_containing(target_dropdown, timeout=30)
    WebDriverWait(driver, 40).until(EC.presence_of_element_located(target_dropdown))

    # --- Select 'Datewise' (Select2 robust) ---
    print("Selecting Datewise…")
    try:
        select2_container = WebDriverWait(driver, 20).until(
            EC.element_to_be_clickable((By.XPATH, "//span[@id='select2-Sdatewise-container']"))
        )
        select2_container.click()
        # use select2 search + Enter
        search_box = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//input[contains(@class,'select2-search__field')]"))
        )
        search_box.clear()
        search_box.send_keys("Datewise")
        time.sleep(0.3)
        search_box.send_keys(Keys.ENTER)
    except TimeoutException:
        # fallback: direct LI
        try:
            datewise_option = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, "//li[contains(@id,'select2-Sdatewise') and normalize-space()='Datewise']"))
            )
            js_click(datewise_option)
        except Exception as e:
            print("Select2 fallback failed:", e)

    # --- Dates ---
    first_day = datetime.today().replace(day=1).strftime("%d/%m/%Y")
    yesterday = (datetime.today() - timedelta(days=1)).strftime("%d/%m/%Y")
    print(f"Setting dates: {first_day} → {yesterday}")

    from_input = find_first([
        (By.ID, "Sdate"),
        (By.NAME, "DateFrom"),
        (By.XPATH, "//input[@placeholder='From' or @placeholder='Date From']")
    ])
    to_input = find_first([
        (By.ID, "dateto"),
        (By.NAME, "DateTo"),
        (By.XPATH, "//input[@placeholder='To' or @placeholder='Date To']")
    ])

    set_date_input(from_input, first_day)
    set_date_input(to_input, yesterday)
    time.sleep(0.5)

    # --- Download Now -> Master Export ---
    print("Triggering download…")
    download_dropdown_locators = [
        (By.XPATH, "//a[contains(@class,'dropdown-toggle')][contains(.,'Download Now')]"),
        (By.XPATH, "//button[contains(@class,'dropdown-toggle')][contains(.,'Download Now')]"),
    ]
    for loc in download_dropdown_locators:
        try:
            safe_click(loc)
            break
        except Exception:
            continue
    else:
        raise TimeoutException("Could not open 'Download Now' dropdown.")

    master_export_locators = [
        (By.XPATH, "//button[@type='submit' and contains(@class,'dropdown-item') and @value='Export' and contains(.,'Master Export')]"),
        (By.XPATH, "//button[contains(.,'Master Export') and (@type='submit' or @value='Export')]"),
        (By.XPATH, "//a[contains(.,'Master Export')]"),
    ]
    for loc in master_export_locators:
        try:
            safe_click(loc)
            break
        except Exception:
            continue
    else:
        driver.save_screenshot("export_button_not_found.png")
        raise TimeoutException("Could not click Master Export.")

    # --- Wait for file ---
    downloaded = wait_for_download(DOWNLOAD_DIR, timeout=240)
    if downloaded:
        print(f"✅ Download complete: {downloaded}")
    else:
        raise TimeoutError("Timed out waiting for the export file to download.")

except Exception as e:
    print("❌ Automation failed:", e)
finally:
    try:
        driver.quit()
    except Exception:
        pass

Opening site…
Logging in…
Opening Data Management…
Opening Attendance…
Opening Monthly Attendance…
Selecting Datewise…
Setting dates: 01/11/2025 → 04/11/2025
Triggering download…
✅ Download complete: C:\Users\Administrator\LIS\MISAutomation\downloads\EmployeeAttendanceMasterReport_05-11-2025.xlsx
